### DEPENDENCIES

In [32]:
with open('libs.txt', 'r') as file:
    for line in file:
        line = line.strip()
        if line:
            exec(f"import {line}")
            
from utils import resample_audio

# RUN

### Load Tokenizers

In [33]:
# from Tokenizers import TokenizersConfig, Tokenizers

# def infer_token(audio_path, checkpoint_path):
#     # load the tokenizer checkpoints
#     checkpoint = torch.load(checkpoint_path)

#     cfg = TokenizersConfig(checkpoint['cfg'])
#     BEATs_tokenizer = Tokenizers(cfg)
#     BEATs_tokenizer.load_state_dict(checkpoint['model'])
#     BEATs_tokenizer.eval()

#     audio_input_16khz = resample_audio(audio_path)
#     labels = BEATs_tokenizer.extract_labels(audio_input_16khz, padding_mask=None)
#     return labels

# labels = infer_token('audios/ex_baby.wav', 'checkpoints/Tokenizer_iter3_plus_AS2M.pt')

### Load Pre-trained

In [34]:
# from BEATs import BEATs, BEATsConfig

# def infer_pretrained(audio_path, checkpoint_path):
#     # load the pre-trained checkpoints
#     checkpoint = torch.load(checkpoint_path)

#     cfg = BEATsConfig(checkpoint['cfg'])
#     BEATs_model = BEATs(cfg)
#     BEATs_model.load_state_dict(checkpoint['model'])
#     BEATs_model.eval()

#     audio_input_16khz = resample_audio(audio_path)
#     representation = BEATs_model.extract_features(audio_input_16khz, padding_mask=None)[0]
#     return representation
    
# representation = infer_pretrained('audios/ex_baby.wav', 'checkpoints/BEATs_iter3_plus_AS2M.pt')

### Load Fine-tuned Models

In [35]:
from BEATs import BEATs, BEATsConfig

def infer_finetuned(audio_path, checkpoint_path):
    # load the fine-tuned checkpoints
    checkpoint = torch.load(checkpoint_path)

    cfg = BEATsConfig(checkpoint['cfg'])
    BEATs_model = BEATs(cfg)
    BEATs_model.load_state_dict(checkpoint['model'])
    BEATs_model.eval()

    audio_input_16khz = resample_audio(audio_path)
    probs = BEATs_model.extract_features(audio_input_16khz, padding_mask=None)[0]
    return probs, checkpoint

In [36]:
def topk_labels_prob(probs, checkpoint):
    classes = pd.read_csv('labels/class_labels_indices.csv', index_col='index')

    results = []
    for (top_label_prob, top_label_idx) in zip(*probs.topk(k=2)):
        top_label = [checkpoint['label_dict'][label_idx.item()] for label_idx in top_label_idx]
        # Get classes from AudioSet class labels indices
        tags = []
        for c in top_label:
            tag = classes[classes['mid'] == c]['display_name'].values[0]
            tags.append(tag)
        results.append([tags[0], top_label_prob.tolist()[0], tags[1], top_label_prob.tolist()[1]])
    return results

In [37]:
def infer(audio_path, checkpoint_path):
    probs, checkpoint = infer_finetuned(audio_path, checkpoint_path)
    results = topk_labels_prob(probs, checkpoint)
    return results

### Compare to previous results...

In [38]:
all_labeled = pd.read_csv('labels/all_labeled.csv', index_col='Unnamed: 0')
all_labeled = all_labeled.drop(labels=['Unnamed: 0.1'], axis=1)

In [39]:
def batch_infer(folder:str = 'files', checkpoint_path:str = 'checkpoints', output = {}):
    """_summary_

    Args:
        folder (str, optional): _description_. Defaults to 'files'.
        checkpoint_path (str, optional): _description_. Defaults to 'checkpoints'.
        output (dict, optional): _description_. Defaults to {}.

    Returns:
        dict: _description_
    """
    for name in os.listdir(folder):
        path = os.path.join(folder, name)
        if os.path.isfile(path):
            filename = os.path.basename(path)
            results = infer(path, checkpoint_path)
            output[filename] = results[0]
        elif os.path.isdir(path):
            batch_infer(path, checkpoint_path, output)
    return output

### INFERENCES - WARNING: the following command takes a while (11 min aprox.)

In [40]:
# beats_results = batch_infer('files', 'checkpoints/BEATs_iter3_plus_AS2M_finetuned_on_AS2M_cpt2.pt')

In [41]:
# arranged_dict = [{**{'audio_id': key}, **{f'col_{i+1}': value[i] for i in range(4)}} for key, value in beats_results.items()]

# # Convert the list of dictionaries into a DataFrame
# df_beats_results = pd.DataFrame(arranged_dict)
# cols = {'col_1': 'first_label_beats','col_2': 'first_prob_beats', 'col_3': 'next_label_beats','col_4': 'next_prob_beats'}
# df_beats_results.rename(columns=cols, inplace=True)
# df_beats_results.head()

In [42]:
# merged_inferences = pd.merge(all_labeled, df_beats_results, on='audio_id')
# merged_inferences.head()                             

### Loading inferences from .csv

In [43]:
# merged_inferences.to_csv('merged_inferences.csv')

In [44]:
merged_inferences = pd.read_csv('merged_inferences.csv', index_col=0)
merged_inferences.head()

,audio_id,first_label_hts,first_prob_hts,next_label_hts,next_prob_hts,first_label_res,first_prob_res,next_label_res,next_prob_res,Label_audio_1,Label_audio_2,first_label_beats,first_prob_beats,next_label_beats,next_prob_beats
0,f7924b777e474520a516aa584a41d7e6.wav,mouse_click,2.319037,cow,2.214936,hen,-1.544079,mouse_click,-2.377246,0,0,Speech,0.863339,"Male speech, man speaking",0.312851
1,903786b251994db3af3c454c9ddfc586.wav,hen,2.887828,dog,1.696776,breathing,-3.434196,dog,-2.485531,0,0,Speech,0.912175,Speech synthesizer,0.250158
2,0c670303907b48d682468639f5be5c1c.wav,dog,3.303216,siren,1.861511,water_drops,-2.043841,coughing,-2.101418,0,0,Speech,0.929904,Telephone,0.302890
3,7c2538cae6bb49548cf6f97cf4f6cbad.wav,siren,2.034104,keyboard_typing,1.913578,hen,0.325456,hen,-3.215177,0,0,Speech,0.941915,Telephone,0.321044
4,92e8a42ea6834c82ad2e937c6f9c2edf.wav,dog,4.262393,chirping_birds,1.623316,water_drops,-1.874607,chirping_birds,-3.507891,0,0,Speech,0.965017,Telephone,0.388490


### Metrics

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Preparing ground truth labels for metrics calculation
ground_truth = merged_inferences['Label_audio_1'] + merged_inferences['Label_audio_2']
y_true = [1 if entry != 0 else 0 for entry in ground_truth]

# Preparing predicted labels for metrics calculation
y_pred_1 = [1 if (('Baby' in entry) or entry == 'Cat') else 0 for entry in merged_inferences['first_label_beats']]
y_pred_2 = [1 if (('Baby' in entry) or entry == 'Cat') else 0 for entry in merged_inferences['next_label_beats']]
y_pred = [1 if entry != 0 else 0 for entry in [x + y for x, y in zip(y_pred_1, y_pred_2)]]

def metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    print("Precisión (Accuracy):", accuracy)

    # Calcular la precisión del modelo
    precision = precision_score(y_true, y_pred)
    print("Precisión (Precision):", precision)

    # Calcular el recall del modelo
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # Calcular la puntuación F1 del modelo
    f1 = f1_score(y_true, y_pred)
    print("Puntuación F1:", f1)

    # Obtener la matriz de confusión
    confusion = confusion_matrix(y_true, y_pred)
    print("Matriz de Confusión:")
    print(confusion)

    # Obtener un informe de clasificación detallado
    report = classification_report(y_true, y_pred)
    print("Informe de clasificación:")
    print(report)

metrics(y_true, y_pred)

Precisión (Accuracy): 0.8238636363636364
Precisión (Precision): 1.0
Recall: 0.515625
Puntuación F1: 0.6804123711340206
Matriz de Confusión:
[[112   0]
 [ 31  33]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.78      1.00      0.88       112
           1       1.00      0.52      0.68        64

    accuracy                           0.82       176
   macro avg       0.89      0.76      0.78       176
weighted avg       0.86      0.82      0.81       176

